In [1]:
# !pip install tensorflow-gpu # make sure cuda toolkit and nvidia dependencies/drivers are correctly installed
# !pip install pandas
# !pip install numpy
# !pip install matplotlib
# !pip install mlflow
# !pip install scipy
# !pip install keras

In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from keras.models import Model


from keras.layers import Input, LSTM, Bidirectional, Dense, Masking, Reshape, Activation, Masking
from keras.layers import Conv1D, BatchNormalization, GlobalAveragePooling1D, Permute, Dropout, concatenate

from keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from time import time
from datetime import datetime, timedelta

from pathlib import Path

from ProcessData import get_vol, get_touches, get_labels, get_horizons, load_data, label_series
from ModelArchitecture import generate_mlstmfcn


In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0
2022-03-02 16:58:34.442379: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-02 16:58:34.840507: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /device:GPU:0 with 4177 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1660 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5


In [4]:
dataset_df, names = load_data("../Data/")

In [5]:
BTC_SAMPLE = pd.DataFrame(index = dataset_df.index)
BTC_SAMPLE["Close"] = dataset_df['BTC_USD-5m']
print(BTC_SAMPLE.index)

DatetimeIndex(['2020-01-16 00:00:00', '2020-01-16 00:05:00',
               '2020-01-16 00:10:00', '2020-01-16 00:15:00',
               '2020-01-16 00:20:00', '2020-01-16 00:25:00',
               '2020-01-16 00:30:00', '2020-01-16 00:35:00',
               '2020-01-16 00:40:00', '2020-01-16 00:45:00',
               ...
               '2022-01-15 10:05:00', '2022-01-15 10:10:00',
               '2022-01-15 10:15:00', '2022-01-15 10:20:00',
               '2022-01-15 10:25:00', '2022-01-15 10:30:00',
               '2022-01-15 10:35:00', '2022-01-15 10:40:00',
               '2022-01-15 10:45:00', '2022-01-15 10:50:00'],
              dtype='datetime64[ns]', name='Timestamp', length=209972, freq=None)


In [6]:
labels = label_series(BTC_SAMPLE.Close)

In [7]:
print(labels.index)

DatetimeIndex(['2020-01-16 01:10:00', '2020-01-16 01:15:00',
               '2020-01-16 01:20:00', '2020-01-16 01:25:00',
               '2020-01-16 01:30:00', '2020-01-16 01:35:00',
               '2020-01-16 01:40:00', '2020-01-16 01:45:00',
               '2020-01-16 01:50:00', '2020-01-16 01:55:00',
               ...
               '2022-01-15 09:50:00', '2022-01-15 09:55:00',
               '2022-01-15 10:00:00', '2022-01-15 10:05:00',
               '2022-01-15 10:10:00', '2022-01-15 10:15:00',
               '2022-01-15 10:20:00', '2022-01-15 10:25:00',
               '2022-01-15 10:30:00', '2022-01-15 10:35:00'],
              dtype='datetime64[ns]', name='Timestamp', length=209955, freq=None)


In [8]:
labels.to_csv('./Processed/BTClabels.csv')

In [9]:
# labels = pd.read_csv('./Processed/BTClabels.csv')
# labels.set_index('Timestamp', inplace=True)

In [10]:
print(BTC_SAMPLE.head(5))
print(BTC_SAMPLE.shape)
print("Price Index 0: {a} vs. Label Index 0: {b}".format(a=BTC_SAMPLE.index[0], b=labels.index[0]))
print("Price Index -1: {a} vs. Price Index -1: {b}".format(a=BTC_SAMPLE.index[-1], b=labels.index[-1]))


                       Close
Timestamp                   
2020-01-16 00:00:00  8845.70
2020-01-16 00:05:00  8844.44
2020-01-16 00:10:00  8824.01
2020-01-16 00:15:00  8812.22
2020-01-16 00:20:00  8754.02
(209972, 1)
Price Index 0: 2020-01-16 00:00:00 vs. Label Index 0: 2020-01-16 01:10:00
Price Index -1: 2022-01-15 10:50:00 vs. Price Index -1: 2022-01-15 10:35:00


In [11]:
BTC_SAMPLE = BTC_SAMPLE.truncate(before=pd.Timestamp(labels.index[0]), \
    after=pd.Timestamp(labels.index[-1]))
labels = labels.truncate(before=pd.Timestamp(BTC_SAMPLE.index[0]), \
     after=pd.Timestamp(BTC_SAMPLE.index[-1]))

np_labels = labels.to_numpy()
one_hot_encoded_labels = to_categorical(np_labels, num_classes=3)
print(labels.value_counts(normalize=True, dropna=True))
print(one_hot_encoded_labels.shape)

 0.0    0.566407
 1.0    0.217828
-1.0    0.215765
Name: label, dtype: float64
(209955, 3)


In [37]:
values = BTC_SAMPLE.Close.to_numpy()
print(values.shape)
if values.ndim == 1:
    values = values.reshape(-1, 1)
scaler = MinMaxScaler()
scaled = scaler.fit_transform(values)
X = scaled


input_timesteps = 128
batch_size = 128

# split into train and test sets (Train: 68%, Val: 12%, Test: 20%)
trainX, testX, trainY, testY = train_test_split(X, one_hot_encoded_labels, test_size=0.20, random_state=42, shuffle = False)
trainX, valX, trainY, valY = train_test_split(trainX, trainY, test_size=0.15, random_state=42, shuffle=False)

train_generator = TimeseriesGenerator(trainX, trainY, length=input_timesteps, sampling_rate=1, batch_size=batch_size)
val_generator = TimeseriesGenerator(valX, valY, length=input_timesteps, sampling_rate=1, batch_size=batch_size)
test_generator = TimeseriesGenerator(testX, testY, length=input_timesteps, sampling_rate=1, batch_size=batch_size)

train_sample_X, train_sample_y = train_generator[0]
test_sample_X, test_sample_y = train_generator[0]

print("TrainX Shape:" + str(trainX.shape))
print("TrainY Shape:" + str(trainY.shape))
print("TestX Shape:" + str(testX.shape))
print("TestY Shape:" + str(testY.shape))
print("Slice of Features from Train Generator:" + str(train_sample_X.shape))
print("Slice of Labels from Train Generator:" + str(train_sample_y.shape))
print("Slice of Features from Test Generator:" + str(test_sample_X.shape))
print("Slice of Labels from Test Generator:" + str(test_sample_y.shape))

(209955,)
TrainX Shape:(142769, 1)
TrainY Shape:(142769, 3)
TestX Shape:(41991, 1)
TestY Shape:(41991, 3)
Slice of Features from Train Generator:(128, 128, 1)
Slice of Labels from Train Generator:(128, 3)
Slice of Features from Test Generator:(128, 128, 1)
Slice of Labels from Test Generator:(128, 3)


In [27]:
def generate_mlstmfcn(max_timesteps, max_features, classes, cells=8):
    # MLSTM-FCN
    # Multivariate Long-Short-Term Memory  
    ip = Input(shape=(max_timesteps, max_features))

    x = Masking()(ip)
    x = LSTM(cells)(x)
    x = Dropout(0.3)(x)

    # y = Permute((2, 1))(ip)
    y = Conv1D(128, 8, padding='same', kernel_initializer='he_uniform')(ip)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    # y = squeeze_excite_block(y)

    y = Conv1D(256, 5, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    # y = squeeze_excite_block(y)

    y = Conv1D(128, 3, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = GlobalAveragePooling1D()(y)

    x = concatenate([x, y])

    out = Dense(classes, activation='softmax')(x)

    model = Model(ip, out)
    model.summary()

    # add load model code here to fine-tune

    return model

In [30]:
units = 128 # Arbituary Number
features = trainX.shape[1] # Number of different trading pairs in the dataframe ie. BTC/USD (aka: # of columns)
num_epoch = 25
learning_rate = 0.00144

logdir = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=logdir, histogram_freq=1)

# input_shape=(train_sample_X[1].shape, train_sample_X[2].shape)
model = generate_mlstmfcn(input_timesteps, features, 3, 64)



adam = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Stop training when a monitored quantity has stopped improving.
callbacks = [EarlyStopping(monitor="loss", \
                            min_delta = 0.00001, \
                            patience = 50, mode = 'auto', \
                            restore_best_weights=True), \
                            tensorboard_callback] 

# Using regression loss function 'Mean Standard Error' and validation metric 'Mean Absolute Error'
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])


# fit network
history = model.fit(train_generator, \
                            epochs=num_epoch, \
                            validation_data=val_generator, \
                            callbacks = callbacks, \
                            verbose=2, \
                            shuffle=False, \
                            initial_epoch=0)


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 128, 1)]     0           []                               
                                                                                                  
 conv1d_9 (Conv1D)              (None, 128, 128)     1152        ['input_4[0][0]']                
                                                                                                  
 batch_normalization_9 (BatchNo  (None, 128, 128)    512         ['conv1d_9[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 activation_9 (Activation)      (None, 128, 128)     0           ['batch_normalization_9[0][

In [31]:
# Return loss value and metric value
score = model.evaluate_generator(test_generator, verbose=0)   
print("CCE: {cce} Accuracy: {acc}".format(cce=score[0], acc=score[1]))

CCE: 36.88494110107422 Accuracy: 0.314645379781723


In [39]:
predictions_array = model.predict(test_generator, callbacks=tensorboard_callback)
pred_df = pd.DataFrame(predictions_array ) #columns=names
pred_labels = pd.DataFrame(testY)
# pred_labels = pred.truncate(after=(pred_df.shape[0] - 1))
print(pred_labels.shape)
print(pred_df.shape)
print(pred_labels.head(20))
print(pred_df.head(20))

(41991, 3)
(41863, 3)
      0    1    2
0   1.0  0.0  0.0
1   1.0  0.0  0.0
2   1.0  0.0  0.0
3   1.0  0.0  0.0
4   1.0  0.0  0.0
5   1.0  0.0  0.0
6   1.0  0.0  0.0
7   1.0  0.0  0.0
8   1.0  0.0  0.0
9   0.0  1.0  0.0
10  0.0  1.0  0.0
11  0.0  1.0  0.0
12  1.0  0.0  0.0
13  0.0  0.0  1.0
14  0.0  0.0  1.0
15  0.0  0.0  1.0
16  1.0  0.0  0.0
17  1.0  0.0  0.0
18  1.0  0.0  0.0
19  1.0  0.0  0.0
               0             1    2
0   7.285609e-11  4.930504e-20  1.0
1   7.142417e-11  4.623306e-20  1.0
2   6.850139e-11  4.271849e-20  1.0
3   6.439358e-11  3.788259e-20  1.0
4   6.105412e-11  3.406776e-20  1.0
5   5.926413e-11  3.055026e-20  1.0
6   5.837638e-11  2.741288e-20  1.0
7   5.781094e-11  2.484647e-20  1.0
8   5.709624e-11  2.341057e-20  1.0
9   5.480858e-11  2.062078e-20  1.0
10  5.048635e-11  1.686181e-20  1.0
11  4.639060e-11  1.399045e-20  1.0
12  4.329815e-11  1.166612e-20  1.0
13  4.205936e-11  1.028923e-20  1.0
14  4.212455e-11  9.462946e-21  1.0
15  4.362334e-11  9.5424

In [ ]:
a = pred_df.tail(100)
b = real_returns_df.tail(100)
i = 1
plt.figure(figsize=(20, 30))
for name in names:
    plt.subplot(len(names), 1, i)
    plt.plot(a[name], color='red')
    plt.plot(b[name], color='green')
    plt.title(name, y=0.5, loc='right')
    i += 1
plt.show()

In [ ]:
%tensorboard --logdir ./logs/